#  RAG의 기본 개념 / 문서 전처리 과정의 이해


---

# 환경 설정 및 준비

In [ ]:
# 필수 라이브러리 설치
# langchain langchain-community langchain-openai
# beautifulsoup4 faiss-cpu langchain-chroma

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

---

## 1. RAG 개념 및 아키텍처

### 1.1 RAG란?

**RAG (Retrieval-Augmented Generation)** 는 기존 LLM의 한계를 보완하기 위한 방법론:

- **문제점**: LLM은 훈련 시점의 고정된 데이터에만 의존
- **해결책**: 외부 지식 베이스를 동적으로 검색하여 응답 생성 시 활용
- **장점**: 최신 정보, 도메인 특화 지식, 사실 기반 응답 가능

### 1.2 RAG의 핵심 구성요소

```mermaid
graph TB
    A[사용자 쿼리] --> B[문서 검색<br/>Retrieval]
    B --> C[관련 문서]
    C --> D[컨텍스트 증강<br/>Augmentation]
    A --> D
    D --> E[LLM 생성<br/>Generation]
    E --> F[최종 응답]
```

**1. 검색 (Retrieval) 시스템**
- 임베딩 모델: 텍스트를 벡터로 변환
- 벡터 데이터베이스: 임베딩 벡터 저장 및 인덱싱
- 유사도 검색: 코사인 유사도, 유클리드 거리 등

**2. 증강 (Augmentation)**
- 검색된 문서 전처리 및 포맷팅
- 프롬프트 엔지니어링
- 컨텍스트 길이 관리

**3. 생성 (Generation)**
- LLM을 통한 최종 응답 생성
- 검색된 컨텍스트와 원본 질의 결합

### 1.3 RAG vs 기존 접근법 비교

| 특성 | 기존 LLM | 파인튜닝 | RAG |
|------|----------|----------|-----|
| 최신 정보 | ❌ | ❌ | ✅ |
| 구현 복잡도 | 낮음 | 높음 | 중간 |
| 계산 비용 | 낮음 | 높음 | 중간 |
| 소스 추적 | ❌ | ❌ | ✅ |
| 환각 방지 | ❌ | △ | ✅ |

---

## 2. 최신 LangChain을 활용한 RAG 구현

### Step 1: **Indexing**

1. 문서 수집 및 전처리
2. 문서 청크 분할
3. 임베딩 생성
4. 벡터 저장소 구축

![RAG Indexing](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png "RAG - Indexing")

`1. 문서 데이터 로드(Load Data)`

- RAG에 사용할 데이터를 불러오는 단계 (검색에 사용될 지식이나 정보)
- 외부 데이터 소스에서 정보를 수집하고, 필요한 형식으로 변환하여 시스템에 로드

In [2]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader  # type: ignore

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Document 객체로 변환
docs = loader.load() 

# 결과 확인
print(f"Document 개수: {len(docs)}")
print(f"Document 길이: {len(docs[0].page_content)}")
print(f"Document 내용: {docs[0].page_content[5000:6000]}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Document 개수: 1
Document 길이: 13271
Document 내용:  좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.

문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.

명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니

In [3]:
docs[0] # Document 객체 확인

Document(metadata={'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}, page_content='\n\n\n\n위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문으로 이동\n\n\n\n\n\n\n\n주 메뉴\n\n\n\n\n\n주 메뉴\n사이드바로 이동\n숨기기\n\n\n\n\t\t둘러보기\n\t\n\n\n대문최근 바뀜요즘 화제임의의 문서로\n\n\n\n\n\n\t\t사용자 모임\n\t\n\n\n사랑방사용자 모임관리 요청\n\n\n\n\n\n\t\t편집 안내\n\t\n\n\n소개도움말정책과 지침질문방\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n보이기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n기부\n\n계정 만들기\n\n로그인\n\n\n\n\n\n\n\n\n개인 도구\n\n\n\n\n\n기부 계정 만들기 로그인\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n목차\n사이드바로 이동\n숨기기\n\n\n\n\n처음 위치\n\n\n\n\n\n1\n최상위 정책\n\n\n\n\n\n\n\n\n2\n\'정책과 지침\'이란?\n\n\n\n\n\n\n\n\n3\n준수\n\n\n\n\n\n\n\n\n4\n집행\n\n\n\n\n\n\n\n\n5\n문서 내용\n\n\n\n\n\n\n\n\n6\n정책과 지침은 백과사전의 일부가 아닙니다\n\n\n\n\n

In [4]:
docs[0].metadata # Document의 메타데이터 확인

{'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8',
 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전',
 'language': 'ko'}

In [5]:
docs[0].page_content # Document의 페이지 내용 확인

'\n\n\n\n위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문으로 이동\n\n\n\n\n\n\n\n주 메뉴\n\n\n\n\n\n주 메뉴\n사이드바로 이동\n숨기기\n\n\n\n\t\t둘러보기\n\t\n\n\n대문최근 바뀜요즘 화제임의의 문서로\n\n\n\n\n\n\t\t사용자 모임\n\t\n\n\n사랑방사용자 모임관리 요청\n\n\n\n\n\n\t\t편집 안내\n\t\n\n\n소개도움말정책과 지침질문방\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n보이기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n기부\n\n계정 만들기\n\n로그인\n\n\n\n\n\n\n\n\n개인 도구\n\n\n\n\n\n기부 계정 만들기 로그인\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n목차\n사이드바로 이동\n숨기기\n\n\n\n\n처음 위치\n\n\n\n\n\n1\n최상위 정책\n\n\n\n\n\n\n\n\n2\n\'정책과 지침\'이란?\n\n\n\n\n\n\n\n\n3\n준수\n\n\n\n\n\n\n\n\n4\n집행\n\n\n\n\n\n\n\n\n5\n문서 내용\n\n\n\n\n\n\n\n\n6\n정책과 지침은 백과사전의 일부가 아닙니다\n\n\n\n\n\n\n\n\n7\n채택 과정\n\n\n\n\n채택 과정 하위섹션 토글하기\n\n\n\n\n\n7.1\n제안과 채택\n\n\n\n\n\n\n\n\n7.2\n내용 변경\n\n\n\n\n\n\n7.2.1\n실질적인 변경\n\n\n\n\n\n\n\n\n\n\n7.3\n격하\n\n\n\n\n\n\n\n\n\n\n8\n같이 보기\n\n\n\n\n\n\n\n\n9\n외부 링크\n\n\n\n\n\n\n\n\

`2. 문서 청크 분할(Split Texts)`

- 불러온 데이터를 작은 크기의 단위(chunk)로 분할하는 과정
- 자연어 처리(NLP) 기술을 활용하여 큰 문서를 처리가 쉽도록 문단, 문장 또는 구 단위로 나누는 작업
- 검색 효율성을 높이기 위한 중요한 과정

    1. 청크 크기 선택
        - 너무 작은 청크: 문맥 손실
        - 너무 큰 청크: 관련성 저하

    2. 중복 영역 설정
        - 문맥 유지를 위해 필요
        - 일반적으로 10-20% 권장

- 설치: pip install langchain_text_splitters 또는 poetry add langchain_text_splitters

In [6]:
# Text Split (Documents -> small chunks: Documents)
from langchain_text_splitters import CharacterTextSplitter  # type: ignore

# 1000자씩 잘라서 200자씩 겹치는 Document로 변환
text_splitter = CharacterTextSplitter(
    separator="\n\n",    # 문단 구분자
    chunk_size=1000,     # 문단 길이
    chunk_overlap=200,   # 겹치는 길이
    length_function=len, # 길이 측정 함수
    is_separator_regex=False,   # separator가 정규식인지 여부
)

splitted_docs = text_splitter.split_documents(docs)

# 결과 확인
print(f"Document 개수: {len(splitted_docs)}")
print("\n\n")

for i, doc in enumerate(splitted_docs):
    print(f"Document {i} 길이: {len(doc.page_content)}")
    print(f"Document {i} 내용: {doc.page_content[:100]}...")
    print("-"*50)

Created a chunk of size 1525, which is longer than the specified 1000
Created a chunk of size 1439, which is longer than the specified 1000


Document 개수: 16



Document 0 길이: 439
Document 0 내용: 위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전

본문으로 이동

주 메뉴

주 메뉴
사이드바로 이동
숨기기

		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로

	...
--------------------------------------------------
Document 1 길이: 993
Document 1 내용: 7.2
내용 변경


7.2.1
실질적인 변경


7.3
격하


8
같이 보기


9
외부 링크


목차 토글

위키백과:정책과 지침

113개 언어


Afrikaansአማርኛ...
--------------------------------------------------
Document 2 길이: 716
Document 2 내용: 링크 편집

프로젝트 문서토론

한국어

읽기원본 보기역사 보기

도구

도구
사이드바로 이동
숨기기

		동작
	


읽기원본 보기역사 보기

		일반
	


여기를 가리키는 문...
--------------------------------------------------
Document 3 길이: 712
Document 3 내용: 콘텐츠 정책(핵심)
중립적 시각
확인 가능
독자 연구 금지
위키백과에 대한 오해

행동 정책
총의
분쟁 해결
편집 분쟁
삭제 정책
차단 정책
법적 위협 금지
인신 공격 금지
문서의...
--------------------------------------------------
Document 4 길이: 993
Document 4 내용: 최상위 정책
 위키백과:위키미디어 정책  문서를 참고하십시오.
위키백과는 비영리 재단인 위키미디어 재단이 운영하고 있으며, 위키미디어 재단은 위키백과의 특정 법적 권리를 가지고 있...
--------------------------------------------------
Document 5 길이: 771
Document 5 내용: 정책

In [7]:
# 글자 수 기준으로 엄격하게 분할하기 
from langchain_text_splitters import CharacterTextSplitter  # type: ignore

# 1000자씩 잘라서 Document로 변환
text_splitter = CharacterTextSplitter(
    separator="",        # 문단 구분자
    chunk_size=1000,     # 문단 길이
    length_function=len, # 길이 측정 함수
    is_separator_regex=False,   # separator가 정규식인지 여부
)

equally_splitted_docs = text_splitter.split_documents(docs)

# 결과 확인
print(f"Document 개수: {len(equally_splitted_docs)}")
print("\n\n")

for i, doc in enumerate(equally_splitted_docs):
    print(f"Document {i} 길이: {len(doc.page_content)}")
    print("-"*50)

Document 개수: 17



Document 0 길이: 996
--------------------------------------------------
Document 1 길이: 999
--------------------------------------------------
Document 2 길이: 1000
--------------------------------------------------
Document 3 길이: 1000
--------------------------------------------------
Document 4 길이: 998
--------------------------------------------------
Document 5 길이: 1000
--------------------------------------------------
Document 6 길이: 999
--------------------------------------------------
Document 7 길이: 1000
--------------------------------------------------
Document 8 길이: 1000
--------------------------------------------------
Document 9 길이: 998
--------------------------------------------------
Document 10 길이: 1000
--------------------------------------------------
Document 11 길이: 1000
--------------------------------------------------
Document 12 길이: 1000
--------------------------------------------------
Document 13 길이: 999
----------------------------------------

`3. 문서 임베딩 생성(Document Embeddings)`

- 임베딩 모델을 사용하여 텍스트를 벡터로 변환
- 임베딩을 기반으로 유사성 검색에 사용
- 임베딩 모델 선택
   - 성능과 비용 고려
   - 다국어 지원 여부 확인

In [8]:
# OpenAI Embeddings - 문장 임베딩

from langchain_openai import OpenAIEmbeddings  # type: ignore

# embedding model 생성
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름을 지정 가능 
)

sample_text = "위키피디아 정책 변경 절차를 알려주세요"
embedding_vector = embedding_model.embed_query(sample_text)
print(f"임베딩 벡터의 차원: {len(embedding_vector)}")
print(f"임베딩 벡터: {embedding_vector[:10]}...")

임베딩 벡터의 차원: 1536
임베딩 벡터: [-0.008893376216292381, 0.04727892577648163, 0.010539451614022255, 0.0014988997718319297, -0.03502481430768967, 0.042317841202020645, -0.0010695199016481638, 0.02194766327738762, 0.0329672209918499, -0.02229059673845768]...


`4. 벡터 저장소 구축 (Vectorstores)`

- 임베딩 벡터를 벡터저장소에 저장 
- 저장된 임베딩을 기반으로 유사성 검색을 수행하는데 활용 

In [9]:
# Chroma 벡터 저장소에 문서 저장하기
from langchain_chroma import Chroma
vector_store = Chroma(embedding_function=embedding_model)

# Document를 VectorStore에 저장
document_ids = vector_store.add_documents(splitted_docs)

# 결과 확인
print(f"저장된 Document 개수: {len(document_ids)}")

저장된 Document 개수: 16


In [10]:
print(f"저장된 Document ID: {document_ids[:5]}...")

저장된 Document ID: ['f47d128e-c3bc-4923-8589-7868118a1df4', 'd52f47e0-ed3f-48b4-a994-b1f6a9f79b13', '809f50d5-2fae-4033-bba9-c68a800c39fa', '6dae1e48-31f2-4fe6-aeeb-ee7ded414af6', '27a7e59b-eaef-4b7a-98a3-4dbf3127ee40']...


In [12]:
# VectorStore에 저장된 Document 개수 확인
#print(f"VectorStore에 저장된 Document 개수: {len(vector_store.store.keys())}")
# ...existing code...
print(f"VectorStore에 저장된 Document 개수: {vector_store._collection.count()}")
#

VectorStore에 저장된 Document 개수: 16


### Step 2: **Retrieval and generation**


![RAG Retrieval and generation](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png "RAG - Retrieval and generation")


`5. 검색 및 생성`

In [13]:
# 벡터 스토어 문서 검색 - 유사도 기반 검색

search_query = "위키피디아 정책 변경 절차를 알려주세요"

results = vector_store.similarity_search(query=search_query, k=2)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")
    print("-"*50)

* 내용 변경
정책과 지침은 다른 위키백과 문서처럼 편집할 수 있습니다. 사전에 변경 사항을 논의하거나 문자화된 총의 기록를 구비할 필요는 없습니다. 그러나 정책과 지침은 민감하고 복잡합니다. 사용자들은 모든 편집 시 공동체의 관점을 충실히 반영하도록, 실수로 새로운 오류 또는 혼동의 원인이 발생하지 않도록 주의해야 합니다.
명심하세요. 정책과 지침의 목적은 대부분의 위키백과인들이 동의하는 바를 진술하는 것이고, 따라서 정책과 지침은 주제에 대한 현재의 총의를 반영해야 합니다. 정책/지침/수필 문서의 편집 자체가 허용되는 관례의 즉각적인 변경을 의미하지는 않습니다. 정책 또는 지침 문서에서 거부된 관행을 권장하는 것은 당연히 나쁜 활동입니다.
아래에서 보듯, 당신은 과감하게 삽입하거나, 토론을 통하여 광범위한 총의를 모으는 방식으로 모범적인 활동 규범을 제시할 수 있습니다. [{'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}]
--------------------------------------------------
* 실질적인 변경
실행하세요. 정책 및 지침 문서를 실질적으로 변경하기 전에, 기존 관행에 대한 합리적인 예외를 설정하는 것이 유용할 수 있습니다. 이러한 방식으로 기존의 관행을 갱신하기 위해, 백:과감과 백:무시 정신에 따라 기존의 관행에서 직접적으로 벗어날 수도 있습니다. 시간이 지나고, 변경에 대한 이의가 없거나 토론을 통해 변경 또는 구현에 대한 광범위한 총의에 도달한 경우, 관행을 설명하는 정책 및 지침 문서를 편집하여 새로운 상황을 반영할 수 있습니다.
토론 먼저 하세요. 토론 문서 내 논의는 일반적으로 정책의 실질적인 변경보다 우선합니다. 이의가 없거

In [ ]:
# 벡터 스토어 검색기 설정 - 유사도 기반 검색

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

# 검색기로 검색하기
results = retriever.invoke(input=search_query)

# 결과 확인
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")
    print("-"*50)

h:\miniconda3\envs\modu\Lib\site-packages\langsmith\client.py:290: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=354a8b6b-fa1b-4857-874c-75619409dc21,id=354a8b6b-fa1b-4857-874c-75619409dc21


* 내용 변경
정책과 지침은 다른 위키백과 문서처럼 편집할 수 있습니다. 사전에 변경 사항을 논의하거나 문자화된 총의 기록를 구비할 필요는 없습니다. 그러나 정책과 지침은 민감하고 복잡합니다. 사용자들은 모든 편집 시 공동체의 관점을 충실히 반영하도록, 실수로 새로운 오류 또는 혼동의 원인이 발생하지 않도록 주의해야 합니다.
명심하세요. 정책과 지침의 목적은 대부분의 위키백과인들이 동의하는 바를 진술하는 것이고, 따라서 정책과 지침은 주제에 대한 현재의 총의를 반영해야 합니다. 정책/지침/수필 문서의 편집 자체가 허용되는 관례의 즉각적인 변경을 의미하지는 않습니다. 정책 또는 지침 문서에서 거부된 관행을 권장하는 것은 당연히 나쁜 활동입니다.
아래에서 보듯, 당신은 과감하게 삽입하거나, 토론을 통하여 광범위한 총의를 모으는 방식으로 모범적인 활동 규범을 제시할 수 있습니다. [{'language': 'ko', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'}]
--------------------------------------------------
* 실질적인 변경
실행하세요. 정책 및 지침 문서를 실질적으로 변경하기 전에, 기존 관행에 대한 합리적인 예외를 설정하는 것이 유용할 수 있습니다. 이러한 방식으로 기존의 관행을 갱신하기 위해, 백:과감과 백:무시 정신에 따라 기존의 관행에서 직접적으로 벗어날 수도 있습니다. 시간이 지나고, 변경에 대한 이의가 없거나 토론을 통해 변경 또는 구현에 대한 광범위한 총의에 도달한 경우, 관행을 설명하는 정책 및 지침 문서를 편집하여 새로운 상황을 반영할 수 있습니다.
토론 먼저 하세요. 토론 문서 내 논의는 일반적으로 정책의 실질적인 변경보다 우선합니다. 이의가 없거

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=354a8b6b-fa1b-4857-874c-75619409dc21,id=354a8b6b-fa1b-4857-874c-75619409dc21
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=2b59ad5e-9bcf-4cdc-b56d-bc7937c2b138,id=2b59ad5e-9bcf-4cdc-b56d-bc7937c2b138; trace=2b59ad5e-9bcf-4cdc-b56d-bc7937c2b138,id=45159eb5-e5bd-4a1f-aec7-b972063685ff; trace=2b59ad5e-9bcf-4cdc-b56d-bc7937c2b138,id=70f2167b-781a-457d-ab22-c9d3752b4805; trace=2b59ad5e-9bcf-4cdc-b56d-bc7937c2b138,id=e90d86cb-941b-4c55-b099-795c10a2f344; trace=2b59ad5e-

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 시스템 프롬프트
system_prompt = (
    "다음 검색된 맥락을 사용하여 사용자의 질문에 답하세요. "
    "답을 모르면 모른다고 하고, 추측하지 마세요. "
    "답변은 한국어로 간결하고 정확하게 작성하세요.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# LLM 모델 설정
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 문서 처리 체인 생성
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# RAG 체인 생성
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# RAG 체인 실행
query = "위키피디아 정책 변경 절차를 알려주세요"

response = rag_chain.invoke({"input": query})

response

{'input': '위키피디아 정책 변경 절차를 알려주세요',
 'context': [Document(id='84fa6f0f-5fe1-43e0-b976-1a0ed5560cbe', metadata={'language': 'ko', 'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전'}, page_content='내용 변경\n정책과 지침은 다른 위키백과 문서처럼 편집할 수 있습니다. 사전에 변경 사항을 논의하거나 문자화된 총의 기록를 구비할 필요는 없습니다. 그러나 정책과 지침은 민감하고 복잡합니다. 사용자들은 모든 편집 시 공동체의 관점을 충실히 반영하도록, 실수로 새로운 오류 또는 혼동의 원인이 발생하지 않도록 주의해야 합니다.\n명심하세요. 정책과 지침의 목적은 대부분의 위키백과인들이 동의하는 바를 진술하는 것이고, 따라서 정책과 지침은 주제에 대한 현재의 총의를 반영해야 합니다. 정책/지침/수필 문서의 편집 자체가 허용되는 관례의 즉각적인 변경을 의미하지는 않습니다. 정책 또는 지침 문서에서 거부된 관행을 권장하는 것은 당연히 나쁜 활동입니다.\n아래에서 보듯, 당신은 과감하게 삽입하거나, 토론을 통하여 광범위한 총의를 모으는 방식으로 모범적인 활동 규범을 제시할 수 있습니다.'),
  Document(id='66e7f34b-b36a-4b52-b269-81371f70f161', metadata={'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'languag

In [16]:
print(response['answer'])

위키피디아 정책 변경 절차는 다음과 같습니다.

1. 토론 우선: 정책이나 지침의 실질적인 변경을 하기 전에 관련 토론 문서에서 충분한 논의를 진행해야 합니다. 토론에서 이의가 없거나 광범위한 총의가 형성되면 변경할 수 있습니다.

2. 사소한 편집은 언제든 가능: 서식, 문법, 명료성 개선과 같은 사소한 편집은 별도의 토론 없이도 할 수 있습니다.

3. 논의 결과 불분명 시 평가: 주요 변경 사항에 대해 토론 결과가 불분명하면 관리자나 독립 편집자가 평가합니다.

4. 지역사회 공표: 중요한 변경은 일반적으로 지역사회에 공표하여 의견을 수렴하는 과정이 필요합니다.

5. 과감한 편집 가능: 기존 논의 기록이 없더라도 정책 문서를 직접 편집할 수 있으나, 변경 후에는 편집 요약이나 토론 문서에 이유를 명확히 밝혀야 하며, 되돌리기 당하면 토론을 통해 의견을 나누는 것이 권장됩니다.

즉, 정책 변경은 토론을 통한 총의 형성이 우선이며, 사소한 편집은 자유롭게 할 수 있고, 중요한 변경은 지역사회와 충분히 소통하는 절차를 거칩니다.


In [17]:
len(response['context'])

2

In [18]:
for doc in response['context']:
    print(f"* {doc.page_content} [{doc.metadata}]")
    print("-"*50)

* 내용 변경
정책과 지침은 다른 위키백과 문서처럼 편집할 수 있습니다. 사전에 변경 사항을 논의하거나 문자화된 총의 기록를 구비할 필요는 없습니다. 그러나 정책과 지침은 민감하고 복잡합니다. 사용자들은 모든 편집 시 공동체의 관점을 충실히 반영하도록, 실수로 새로운 오류 또는 혼동의 원인이 발생하지 않도록 주의해야 합니다.
명심하세요. 정책과 지침의 목적은 대부분의 위키백과인들이 동의하는 바를 진술하는 것이고, 따라서 정책과 지침은 주제에 대한 현재의 총의를 반영해야 합니다. 정책/지침/수필 문서의 편집 자체가 허용되는 관례의 즉각적인 변경을 의미하지는 않습니다. 정책 또는 지침 문서에서 거부된 관행을 권장하는 것은 당연히 나쁜 활동입니다.
아래에서 보듯, 당신은 과감하게 삽입하거나, 토론을 통하여 광범위한 총의를 모으는 방식으로 모범적인 활동 규범을 제시할 수 있습니다. [{'language': 'ko', 'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전'}]
--------------------------------------------------
* 실질적인 변경
실행하세요. 정책 및 지침 문서를 실질적으로 변경하기 전에, 기존 관행에 대한 합리적인 예외를 설정하는 것이 유용할 수 있습니다. 이러한 방식으로 기존의 관행을 갱신하기 위해, 백:과감과 백:무시 정신에 따라 기존의 관행에서 직접적으로 벗어날 수도 있습니다. 시간이 지나고, 변경에 대한 이의가 없거나 토론을 통해 변경 또는 구현에 대한 광범위한 총의에 도달한 경우, 관행을 설명하는 정책 및 지침 문서를 편집하여 새로운 상황을 반영할 수 있습니다.
토론 먼저 하세요. 토론 문서 내 논의는 일반적으로 정책의 실질적인 변경보다 우선합니다. 이의가 없거

# [실습 프로젝트]

### RAG 파이프라인 구축하기

1. 데이터 준비 → 텍스트 처리 → 임베딩 → 검색 → 평가
2. 각 과제마다 구현해야 할 함수와 결과물 정의
3. 단위 테스트를 통한 검증

In [19]:
# 1단계: 데이터 준비 - 웹문서 검색을 위해 관련 URL 가져오기
web_urls = [
    "https://n.news.naver.com/mnews/article/029/0002927209",
    "https://n.news.naver.com/mnews/article/092/0002358620",
    "https://n.news.naver.com/mnews/article/008/0005136824",
]

In [20]:
# 2단계: WebBaseLoader를 사용해 텍스트 로드
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_urls)
docs = loader.load()

print(f"Loaded {len(docs)} documents")

Loaded 3 documents


In [21]:
# 3단계: CharacterTextSplitter로 문서 분할
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
splitted_docs = text_splitter.split_documents(docs)

print(f"Split into {len(splitted_docs)} chunks")

Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of size 1526, which is longer than the specified 1000
Created a chunk of size 1526, which is longer than the specified 1000


Split into 45 chunks


In [22]:
# 4단계: 임베딩 및 벡터 저장소 구현
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(embedding_function=embedding_model)
document_ids = vector_store.add_documents(splitted_docs)

print(f"Added {len(document_ids)} documents to vector store")

Added 45 documents to vector store


In [24]:
# 5단계: RAG 기반 QA 체인 구현
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

system_prompt = (
    "다음 검색된 맥락을 사용하여 사용자의 질문에 답하세요. "
    "답을 모르면 모른다고 하고, 추측하지 마세요. "
    "답변은 한국어로 간결하고 정확하게 작성하세요.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [25]:
# 6단계: QA 체인으로 질문 응답
query = "뉴스 기사에서 주요 내용을 요약해 주세요"
response = rag_chain.invoke({"input": query})

print(response['answer'])

1. 한 개인이 8억 원의 은닉재산을 암호화폐에 숨긴 사실이 적발됨.
2. 거래정지된 종목에서 수천억 원 규모의 매매 사례가 발생.
3. 전북 경찰 간부가 갑질 의혹으로 전보 조치됨.
4. 남편의 불륜 대신 시어머니가 상간녀를 몰래 도와준 사건이 보도됨.
5. 백종원이 'K소스'를 해외로 수출하며 1000억 원 매출을 목표로 하고 있음.
6. 관악구에서 칼부림 사건의 피의자가 구두로 피의사실을 인정함.
